# 🎅 Welcome to the North Pole Vector Search System! 🎄

## The Challenge at the North Pole
Every year, Santa receives **millions of letters** from children around the world. Each letter is unique - some children are polite and formal, others are casual and excited. Some ask for creative gifts like art supplies, while others want the latest tech gadgets. Some letters overflow with Christmas spirit, while others are short and to the point.

**The Problem**: How can Santa's elves quickly find similar letters, identify trends, and match children with the perfect gifts?

## The Solution: Vector Search Magic! ✨
This notebook creates a **semantic search system** using AI-powered vector embeddings. Instead of just matching keywords, our system understands the **meaning** behind each letter:
* **Personality & Tone**: Is the child grateful? Excited? Polite? Casual?
* **Gift Preferences**: Creative items? Tech gadgets? Toys? Unusual requests?
* **Christmas Spirit**: How much holiday enthusiasm does the letter show?
* **Writing Style**: Formal or informal? Detailed or brief?

## What You'll Learn
* How to set up Databricks Vector Search
* How to create embeddings from text data
* How to perform semantic searches using natural language
* Real-world applications for RAG (Retrieval-Augmented Generation)

**Let's help Santa's elves work smarter, not harder!** 🎁

In [0]:
%pip install -U --quiet databricks-sdk==0.49.0 "databricks-langchain>=0.4.0" databricks-agents mlflow[databricks] databricks-vectorsearch==0.55 langchain==0.3.25 langchain_core==0.3.59 bs4==0.0.2 markdownify==0.14.1 pydantic==2.10.1
dbutils.library.restartPython()

## 🎁 Step 0: Configuration Setup

### What's Happening Here?
Before we can start searching through Santa's letters, we need to set up our workspace. Think of this as preparing the workshop before the elves start working!

**We're creating**:
* 📚 **Source Table**: Points to our dataset of letters from Canadian children
* 🔌 **Endpoint Name**: A unique name for our Vector Search endpoint (like naming a new workshop station)
* 📇 **Index Name**: A unique name for our searchable index (like creating a new filing system)

**Why the timestamp?** Each time you run this notebook, it creates a fresh endpoint and index with a unique name. This prevents conflicts if you're experimenting or running multiple versions!

🎯 **Pro Tip**: In production, you'd use fixed names and update existing endpoints rather than creating new ones each time.

In [0]:
# 🎄 North Pole Vector Search System
# This notebook creates vector embeddings for letters to Santa to enable semantic search
# Elves can find similar letters, discover trends, and match children with perfect gifts!

import time
import datetime
from databricks.vector_search.client import VectorSearchClient

# Configuration
source_table_name = '12daysofdemos.raw_data.holiday_letters_canada'

# Create unique names with timestamp for this magical vector search system
timestamp = str(int(time.time()))
endpoint_name = f'santa_letters_endpoint_{timestamp}'
index_name = f'12daysofdemos.raw_data.santa_letters_index_{timestamp}'

print("🎅 North Pole Vector Search Configuration")
print("="*60)
print(f"📚 Source Table: {source_table_name}")
print(f"🔌 Endpoint Name: {endpoint_name}")
print(f"📇 Index Name: {index_name}")
print("="*60)

## ✨ Understanding Vector Embeddings: The Magic Behind Semantic Search

### What Are Vector Embeddings?
Imagine if every letter to Santa could be represented as a point in a magical multi-dimensional space, where similar letters are close together and different letters are far apart. That's exactly what vector embeddings do!

**Traditional Search** (Keyword Matching):
* Search for "grateful" → Only finds letters with the word "grateful"
* Misses letters saying "thankful", "appreciate", "thank you so much"

**Vector Search** (Semantic Understanding):
* Search for "grateful" → Finds ALL letters expressing gratitude
* Understands synonyms, context, and meaning
* Groups similar concepts together automatically

### How It Works for Santa's Letters
1. 📝 **Input**: Each letter's full text (personality, tone, gift requests)
2. 🧠 **AI Model**: Databricks BGE (BAAI General Embedding) model reads and understands the text
3. 📊 **Output**: A vector (list of numbers) that captures the letter's meaning
4. 🔍 **Search**: Find letters with similar vectors = similar meanings!

### Real-World Example
**Letter A**: "Dear Santa, I've been really good this year. I'd love some art supplies!"
**Letter B**: "Hi Santa! I've tried my best to behave. Could I get painting materials?"
**Letter C**: "Yo Santa! I want a PlayStation 5 and a drone!"

➡️ Letters A and B will have **similar vectors** (polite, creative gifts)
➡️ Letter C will have a **different vector** (casual, tech gifts)

**This is the magic that lets elves find patterns Santa never knew existed!** 🎅✨

## 🎅 Step 1: Create Vector Search Endpoint

⚡ **IMPORTANT**: Spinning up a Vector Search endpoint can take 20-30 minutes to provision! ⏳

This endpoint will power our semantic search system, allowing Santa's elves to find letters based on:
* **Personality traits**: grateful, excited, polite, enthusiastic
* **Gift categories**: creative gifts, tech gadgets, toys, outdoor items
* **Christmas spirit**: holiday enthusiasm, festive language, kindness
* **Special requests**: unusual items, specific brands, multiple wishes

In [0]:
# Initialize Vector Search Client
vsc = VectorSearchClient(disable_notice=True)

print(f"🎄 Creating new endpoint: {endpoint_name}")
print(f"🎄 Creating new index: {index_name}")
print("\n⏳ This may take 20-30 minutes... Time for some hot cocoa! ☕\n")

# Create the vector search endpoint and wait for it to be online
try:
    endpoint = vsc.create_endpoint_and_wait(
        name=endpoint_name, 
        endpoint_type='STANDARD',
        verbose=True,
        timeout=datetime.timedelta(seconds=3600)
    )
    print(f"\n✅ Successfully created endpoint: {endpoint_name}")
except Exception as e:
    print(f"\n❌ Error creating endpoint: {e}")
    raise

## 📇 Step 2: Creating the Vector Search Index

### What's Happening in This Step?
Now that our endpoint is ready, we need to create an **index** - think of it as Santa's magical filing system that organizes all the letters by their meaning!

### Key Components:

**1. Clone the Source Table** 📋
* We create a copy of the original letters table
* This protects the original data from any changes
* Enables Change Data Feed (CDF) for real-time updates

**2. Enable Change Data Feed** 🔄
* Tracks any changes to the table (new letters, updates)
* Allows the index to stay synchronized automatically
* Essential for production systems that receive new letters daily

**3. Create the Index** 🎯
* **Primary Key**: `name` (each child's name identifies their letter)
* **Embedding Source**: `letter` column (the full text we want to search)
* **Embedding Model**: `databricks-bge-large-en` (the AI that understands the text)
* **Pipeline Type**: `TRIGGERED` (we control when to update the index)

### Why the `letter` Column?
The `letter` column contains the **complete text** of each child's letter, including:
* 👶 Personality and tone
* 🎁 Gift requests and preferences  
* 🎄 Christmas spirit and enthusiasm
* ✍️ Writing style (formal vs casual)

By embedding this full text, we capture **all the nuances** that make each letter unique!

⏳ **Note**: The index creation starts immediately, but building embeddings for all letters takes a few minutes.

In [0]:
# Clone the source table to avoid affecting the original
cloned_table_name = "12daysofdemos.raw_data.bronze_holiday_letters_canada"
spark.sql(f"""
CREATE OR REPLACE TABLE {cloned_table_name} CLONE {source_table_name}
""")

# Enable Change Data Feed (CDF) on the cloned table
spark.sql(f"""
ALTER TABLE {cloned_table_name}
SET TBLPROPERTIES (delta.enableChangeDataFeed = true)
""")

# Create the vector search index using BGE embeddings on the 'letter' column
try:
    index = vsc.create_delta_sync_index(
        endpoint_name=endpoint_name,
        index_name=index_name,
        source_table_name=cloned_table_name,
        pipeline_type="TRIGGERED",
        primary_key="name",
        embedding_source_column='letter',
        embedding_model_endpoint_name='databricks-bge-large-en'
    )
    print(f"\n✅ Successfully created index: {index_name}")
    print("\n🎉 Vector search setup complete!")
    print(f"\n🔌 Endpoint: {endpoint_name}")
    print(f"📇 Index: {index_name}")
    print("\n🎅 The elves can now search for letters semantically!")
except Exception as e:
    print(f"\n❌ Error creating index: {e}")
    raise

## 🎅 Step 3: Test Semantic Search!

Time to put our North Pole Vector Search to the test! We'll search for letters based on:
* **Personality & tone**: grateful, excited, polite, casual
* **Gift preferences**: creative gifts, tech gadgets, unusual requests
* **Christmas spirit**: holiday enthusiasm, festive language

**How it works**: Describe what you're looking for in natural language, and the AI finds matching letters - even if the exact words aren't in the text!

In [0]:
# Wait for the index to finish building embeddings
print(f"🎄 Testing vector search with index: {index_name}")
print("⏳ Adding a 100-second buffer to make sure the index is up and running...")
print("🎅 Perfect time to check your nice list twice!\n")
time.sleep(100)
print("✅ Ready to search!")

In [0]:
# Test 1: Find letters from grateful children asking for creative gifts
print("🔍 Searching for: Grateful children asking for creative gifts...\n")

try:
    results1 = spark.sql(f"""
    SELECT 
        search_score,
        name,
        city,
        province,
        letter,
        gifts
    FROM VECTOR_SEARCH(
        index => '{index_name}',
        query_text => 'grateful thankful children asking for creative artistic unique gifts',
        num_results => 5
    )
    ORDER BY search_score DESC
    """)
    
    print("✅ Vector search is working!")
    print("🎄 Top 5 letters from grateful children with creative requests:\n")
    display(results1)
    
except Exception as e:
    print(f"⚠️ Vector search not ready yet: {e}")
    print("🕒 The index is still building embeddings. Wait a few more minutes!")

### 💡 What Did We Learn from Search 1?

**Query**: "grateful thankful children asking for creative artistic unique gifts"

**What the AI Found**:
* Letters expressing gratitude (even without the exact word "grateful")
* Requests for creative items like art supplies, LEGO sets, craft materials
* Children who took time to be polite and appreciative

**Why This Matters for Santa's Workshop**:
* 🎁 **Gift Planning**: Bulk order art supplies for grateful, creative kids
* ⭐ **Priority Delivery**: Reward especially thankful children
* 📊 **Trend Analysis**: See which creative gifts are most popular

**The Magic**: Notice how the search found letters that express gratitude in different ways - "thank you", "appreciate", "grateful", or just a polite tone. That's semantic search in action! ✨

In [0]:
# Test 2: Find letters with high Christmas spirit and unusual requests
print("🔍 Searching for: Letters with high Christmas spirit and unusual requests...\n")

try:
    results2 = spark.sql(f"""
    SELECT 
        search_score,
        name,
        city,
        province,
        letter,
        gifts
    FROM VECTOR_SEARCH(
        index => '{index_name}',
        query_text => 'excited enthusiastic festive Christmas spirit unusual unique interesting gift requests',
        num_results => 5
    )
    ORDER BY search_score DESC
    """)
    
    print("🎄 Top 5 letters with high Christmas spirit and unusual requests:\n")
    display(results2)
    
except Exception as e:
    print(f"⚠️ Vector search not ready yet: {e}")

### 💡 What Did We Learn from Search 2?

**Query**: "excited enthusiastic festive Christmas spirit unusual unique interesting gift requests"

**What the AI Found**:
* Letters with high energy and excitement ("hyped", "can't wait", exclamation marks!!!)
* Unusual gift requests like magnetic levitating globes, solar-powered chargers
* Strong Christmas spirit and festive language

**Why This Matters for Santa's Workshop**:
* 🎄 **Christmas Spirit Award**: Identify the most festive letters
* 🚀 **Innovation Tracking**: Spot trending unique gifts before they go mainstream
* 🎯 **Special Attention**: Unusual requests might need custom workshop solutions

**The Magic**: The AI understands enthusiasm through writing style, not just words. It picks up on tone, punctuation, and energy level! 🎉

## 🎄 North Pole Vector Search System - Complete! 🎉

### 🎅 What We Built
We created a semantic search system that captures:
* **Personality traits**: gratitude, excitement, politeness, enthusiasm
* **Gift preferences**: creative items, tech gadgets, toys, unusual requests
* **Christmas spirit**: festive language, holiday enthusiasm, kindness
* **Writing style**: formal vs casual, detailed vs brief

### 🎁 How Elves Can Use This

**Gift Matching**
* Find children with similar interests for bulk gift preparation
* Match unusual requests with creative solutions
* Identify tech-savvy kids for electronic gifts

**Trend Discovery**
* Discover popular gift categories across regions
* Identify emerging toy trends
* Find unique or unusual gift requests

**Quality Control**
* Prioritize letters from especially grateful children
* Find letters showing exceptional Christmas spirit
* Identify children who've been particularly good

**RAG Applications**
* Use as context for AI-powered gift recommendations
* Generate personalized responses to letters
* Create gift suggestion reports for Santa

### ✨ The Magic of Vector Embeddings
Instead of keyword matching, our system understands **meaning**. Search for "grateful" and find letters that express thanks in many different ways. Search for "creative gifts" and discover art supplies, building sets, and craft kits - even if those exact words weren't used!

**Merry Christmas from the North Pole! 🎅🎄**